# Statistical machine learning - Notebook 4, version for students
**Author: Michał Ciach**  
Exercises denoted with a star \* are optional. They may be more difficult or time-consuming.  


## Data & library imports

In [1]:
!pip install gdown

In [2]:
!gdown https://drive.google.com/uc?id=1xOJfD-jexDbHSOCg1EiyAxqc5kXjMvX0
!gdown https://drive.google.com/uc?id=1y5NKR3aWB0DbAuSWcg6ffa1Atu2unpOA

zsh:1: no matches found: https://drive.google.com/uc?id=1xOJfD-jexDbHSOCg1EiyAxqc5kXjMvX0
zsh:1: no matches found: https://drive.google.com/uc?id=1y5NKR3aWB0DbAuSWcg6ffa1Atu2unpOA


In [3]:
# !pip install --upgrade scipy

In [4]:
import pandas as pd
import numpy as np
import plotly.express as px
from scipy.stats import t as tstud
from scipy.stats import ttest_ind, ttest_rel, ttest_1samp, norm, kstest, mannwhitneyu, shapiro, chisquare, chi2_contingency
import plotly.graph_objects as go
from statsmodels.stats.multitest import fdrcorrection

In [5]:
protein_lengths = pd.read_csv('protein_lengths.tsv', sep='\t')
protein_lengths['LogLength'] = np.log10(protein_lengths['Protein length'])
protein_lengths

,Scientific name,Common name,Protein ID,Protein length,LogLength
0,Homo sapiens,Human,NP_000005.3,1474,3.168497
1,Homo sapiens,Human,NP_000006.2,290,2.462398
2,Homo sapiens,Human,NP_000007.1,421,2.624282
3,Homo sapiens,Human,NP_000008.1,412,2.614897
4,Homo sapiens,Human,NP_000009.1,655,2.816241
...,...,...,...,...,...
648731,Imleria badia,Bay bolete (mushroom),KAF8560453.1,494,2.693727
648732,Imleria badia,Bay bolete (mushroom),KAF8560454.1,737,2.867467
648733,Imleria badia,Bay bolete (mushroom),KAF8560455.1,554,2.743510
648734,Imleria badia,Bay bolete (mushroom),KAF8560456.1,813,2.910091


In [6]:
human_protein_lengths = protein_lengths.loc[protein_lengths['Common name'] == 'Human'].copy()
# Note: without .copy(), some versions of Pandas may return a View.
# This may interfere with adding a new column to human_protein_lengths.
print(human_protein_lengths.head())
print()
print(human_protein_lengths.describe())

  Scientific name Common name   Protein ID  Protein length  LogLength
0    Homo sapiens       Human  NP_000005.3            1474   3.168497
1    Homo sapiens       Human  NP_000006.2             290   2.462398
2    Homo sapiens       Human  NP_000007.1             421   2.624282
3    Homo sapiens       Human  NP_000008.1             412   2.614897
4    Homo sapiens       Human  NP_000009.1             655   2.816241

       Protein length      LogLength
count   136193.000000  136193.000000
mean       692.655775       2.711540
std        746.993628       0.329892
min         12.000000       1.079181
25%        316.000000       2.499687
50%        514.000000       2.710963
75%        842.000000       2.925312
max      35991.000000       4.556194


In [7]:
citizen_incomes = pd.read_csv('citizen incomes.tsv', sep='\t')
citizen_incomes

,Country,Income
0,A,43806.926920
1,A,62825.327197
2,A,46660.685387
3,A,54625.879014
4,A,41976.428035
...,...,...
1995,B,55338.284370
1996,B,95699.796316
1997,B,34417.930506
1998,B,20637.868432


## Multiple hypothesis testing



When performing statistical tests, we reject the null hypothesis when the p-value is below a set threshold, traditionally 0.05. A consequence of this approach is that 5% of positive results will be false positives (Type I errors; incorrect rejections of a true null hypothesis), which may be a huge number when thousands of tests are performed in large-scale studies.  

In order to limit the number of false positives, we use *multiple testing corrections*. One of the most useful ones is the Benjamini-Hochberg correction, which controls the False Discovery Rate (FDR), i.e. the proportion of false positives among all positive results. Note the difference from significance levels: the false discovery rate is the proportion of false positives among all positives in the *results* of the tests, while the significance level is the proportion of false positives among the *true negatives* (cases when $H_0$ is true).  

The assumptions and details of the Benjamini-Hochberg correction (and other common corrections) were discussed in the lecture; you may want to review them now. One of the most important assumptions is that the p-values come from a set of independent tests.  

A common way to perform the Benjamini-Hochberg correction is to transform the p-values $p_1, \dots, p_m$ in a way to obtain so-called *q-values* $q_1, \dots, q_m$, such that we have an FDR on the level of $Q$ when we accept all hypotheses $H_i$ with $q_i \leq Q$.


Do not confuse the False Discovery Rate (FDR) with other similarly named metrics, like the False Positive Rate (FPR)! FDR is the ratio of the number of false positives to the number of all reported positives, while FPR is the ratio of the number of false positives to the number of true null hypotheses. You can see a list of other common metrics [here](https://en.wikipedia.org/wiki/Confusion_matrix).   

In the next exercise, we will use *stochastic simulations* to generate our own data. First, however, let's talk about what it means to simulate a random variable.  

*Theory time.* The term *simulating a random variable* is confusing for many students, especially the ones that paid attention in probability classes. Recall that a random variable $X$ is a function from the space of elementary outcomes $\Omega$ to the space of real (or integer) numbers. When some students hear "let's simulate a sequence of random variables $X_1, \dots, X_n$, they wonder: how do we simulate a sequence of functions? Well, this is not what we mean at all.   

We define random variables as functions because we want to have a mathematical model of randomness. This is a convenient way to rigorously define what it means that a variable $X$ takes some value with some probability. For example, we can use it to formally define $\mathbb{P}(X=1) = \mathbb{P}(\{ \omega: X(\omega)  =1 \})$. We need this definition, because the concept of probability is defined on the space $\Omega$, not on the real numbers. However, this is not how we interpret or use random variables in practice.

Recall that the number of dots on a set of dice is a sequence of random variables $X_1, \dots, X_n$ before you throw the dice. When you throw them, you get realizations of the random variables, $X_1(\omega), \dots, X_n(\omega)$, which are a sequence of numbers (and often called *random numbers*). Measuring the outcome of a random experiment corresponds to fixing $\omega$ in a sequence of random variables. This is why we can interpret random variables as numerical variables. Defining them as functions is necessary only because otherwise terms like $\mathbb{P}(X=1)$ don't have any mathematical meaning.

"Fixing $\omega$" is also what we mean by simulating a sequence of random variables: we want to get a sequence of numbers $x_1, \dots, x_n$ that are realizations of a sequence of random variables $X_1, \dots X_n$, so that $x_i = X_i(\omega)$. However, simulating doesn't mean that we pick $\omega$ by hand. Remember that $\omega$ is mostly a mathematical model, not a physical being. Instead of selecting it by hand, we rely on random number generators that "fix $\omega$" for us.   

You have already encountered a few random number generators. A set of dice is one of them. When you throw the dice, you can imagine that the action of throwing corresponds to fixing $\omega$ in a sequence of random variables - and you get a sequence of numbers as a result. You probably have also used pseudo-random number generators in your programs. These are algorithms that return pseudo-random numbers between 0 and 1. When you run this algorithm, you can imagine that this corresponds to fixing $\omega$ in a random variable $X\sim U([0, 1])$ - and you get a random number as a result.   

These two random number generators (the dice and the algorithm) return numbers with very simple distributions. However, we can simulate random numbers with many other distributions, such as the Normal one. There are many techniques to do this, which usually rely on transforming random numbers distributed uniformly between 0 and 1 (formally speaking, numbers which are a realization of a sequence of random variables $X_i$ such that $X_i \sim_{iid} U([0, 1])$). In this course, you don't need to worry about these techniques: we will simulate random variables using algorithms which are already implemented in Python libraries.



**Exercise 1.** In this exercise, we'll learn how to use the Benjamini-Hochberg correction to control the False Discovery Rate, as well as how to simulate our own data sets. We will generate a set of samples from two different distributions and test for the value of their means, resulting in a set of tests in which some null hypotheses are true, and some are false.  

1. Generate $M=1000$ random samples of size $N=10$ from the standard normal distribution, and $M=1000$ samples of size $N=10$ from the normal distribution with expected value $\mu=1$. Use the `norm.rvs` function from `scipy` (if you need to, you can look for its documentation online).
2. For each sample, do a one-sample Student's t test to check whether the mean is equal zero. Save the p-values.      
  2.1. How many null hypotheses are true, and how many are false?   
3. Reject the null hypotheses on the significance level 5% (i.e. when the p-values are less than 0.05).   
  3.1. How many positive results (i.e. rejections of the null hypothesis) did you get?  
  3.2. How many true positives did you get?  
  3.3. How many false positives did you get?  
  3.4. How many false positives did you expect given the significance level?   
  3.5. Calculate the obtained False Positive Rate. Is it close to the significance level? Why/why not?       
  3.6. Calculate the obtained False Discovery Rate.     
  3.7. Which of the previous points depend on $N$? Which do depend on $M$? Give a theoretical argument.   
4. Perform the Benjamini-Hochberg correction on your p-values using the `fdrcorrection` function from the `statsmodels` library. Reject the null hypothesis at the FDR level 10%.  
  4.1. How many positive results did you get?  
  4.2. How many true positives did you get?  
  4.3. How many false positives did you get?  
  4.4. Calculate the obtained False Positive Rate. Is it close to the significance level? Why/why not?     
  4.5. Calculate the obtained False Discovery Rate. Is it close to the assumed 10% FDR? Why/why not?  
  4.6. How many false positives did you expect given the FDR level?  
5. Suppose all the samples were drawn from the standard normal distribution. What would be the effect of applying the Benjamini-Hochberg correction? How many false positives would you expect to obtain?    
6. \* Based on the description of the Benajmini-Hochberg procedure from [this Wikipedia article](https://en.wikipedia.org/wiki/False_discovery_rate#Benjamini%E2%80%93Hochberg_procedure), figure out how to compute the q-values given a list of p-values. Write a function that accepts a vector of p-values and returns the corresponding q-values. Compare your results to the `fdrcorrection` function from the `statsmodels` library on a p-value vector (0.01, 0.1, 0.01, 0.2, 0.01, 0.1).

In [8]:
import numpy as np
from scipy.stats import ttest_1samp
from scipy.stats import norm
from statsmodels.stats.multitest import fdrcorrection

In [9]:
# 1
M = 1000
N = 10

samples_standard = norm.rvs(size=(M, N))
samples_mean_1 = norm.rvs(loc=1, size=(M, N))

In [10]:
def ttest(dataset):
    return np.array([ttest_1samp(sample, 0).pvalue for sample in dataset])

def calculate_stats(p_values, alpha = 0.05):
    reject = np.array(p_values) < alpha
    true_positives = np.sum(reject)
    false_positives = np.sum(reject)
    # 3.4,5,6
    expected_false_positives = M * alpha
    fpr = false_positives / M
    fdr = false_positives / max(1, true_positives)
    tpr = true_positives / M
    return tpr, fpr, fdr

In [11]:
# 2
p_values_standard = ttest(samples_standard)
p_values_mean_1 = ttest(samples_mean_1)

In [12]:
# 3
_, fpr, fdr = calculate_stats(p_values_standard)
_, fpr, fdr = calculate_stats(p_values_mean_1)

In [13]:
# 4
_, p_values_standard = fdrcorrection(p_values_standard, alpha=0.1)
_, p_values_mean_1 = fdrcorrection(p_values_mean_1, alpha=0.1)

_, fpr, fdr = calculate_stats(p_values_standard, alpha=0.1)
_, fpr, fdr = calculate_stats(p_values_mean_1, alpha=0.1)

ValueError: not enough values to unpack (expected 4, got 2)

In [ ]:
# Since all samples are drawn from the standard normal distribution, no true positives are expected
# Applying Benjamini-Hochberg correction should result in zero false positives.



In [ ]:
def q_values(p_values):
    sorted_indices = np.argsort(p_values)
    ranked_p_values = p_values[sorted_indices]
    q_values = alpha * np.arange(1, len(p_values) + 1) / len(p_values)
    return q_values[np.argsort(sorted_indices)]

In [ ]:
# Example p-values vector
example_p_values = np.array([0.01, 0.1, 0.01, 0.2, 0.01, 0.1])

# Compute q-values using the custom function
example_q_values_custom = q_values(example_p_values)

# Compute q-values using statsmodels fdrcorrection function
_, example_q_values_statsmodels, _, _ = fdrcorrection(example_p_values, alpha=0.1)


**Exercise 2.\*** In the previous exercise, you may have noticed that the results vary between the runs. Each sampling of the $2M$ samples returns a different FDR. In this exercise, we'll analyze the distribution of the FDR.  
1. Repeat the calculation of the "raw" FDR (i.e. before the correction) and the "BH" FDR (after the correction) for $R=1000$ times. Also calculate the True Positive Rate, i.e. the fraction of correctly identified positives.  
2. Generate histograms that compare the FDR and TPR before and after the BH correction.     
  2.1. Is there any adverse effect to using the Benjamini-Hochberg correction?  
3. Calculate the average FDR after the correction. Is it close to the assumed 10% level? Why/why not?       

In [ ]:
# 1
R = 1000 
raw_fdr = []
bh_fdr = []
tpr = []

alpha = 0.05

for _ in range(R):
    samples_standard = np.random.randn(M, N)
    samples_mean_1 = np.random.normal(loc=1, size=(M, N))
    tpr_sample, fdr_sample, bh_sample = 
    
    

**Exercise 3.** In each of the research experiments below, there is either some procedure applied incorrectly, or it is correct but can be improved. For each point, say whether all the assumptions are satisfied, and whether the methodology can be improved (e.g. by using a different statistical test or correction procedure). If the assumptions are violated, discuss the possible effect of this kind of violation. For extra points, support your claims with theoretical arguments or simulations.        
1. A researcher studies the effect of glucose consumption on rat well-being. The researcher has partitioned $N=20$ rats into two groups: an experimental one with excess glucose in the diet, and a control one with a normal diet. After a week, the researcher has measured the weight of the rats, the length of sleep, the blood glucose level, the abdominal circumference, the body fat percentage, and the anxiety level (using the elevated plus maze assay). The researcher has compared each factor between the experimental and the control group using the two-sample Student's t test for unpaired observations, previously verifying that the compared variables are normally distributed using the Shapiro-Wilk's test. Next, the researcher has corrected the Student's t test p-values for multiple testing using the Benjamini-Hochberg correction at FDR level 20%.   
2. A researcher studies the effect of consumption of small amounts of vinegar on the increase of the blood glucose level (BGC) in mice. The researcher has partitioned $N=20$ mice into equally-sized experimental and control groups. At the beginning of the experiment, he has measured the BGC of all mice. Next, for four days, mice from both group were given a diet enriched in glucose. Mice from the experimental groups were also given small amounts of vinegar in their diet. After four days, the BGC of all mice was measured again. Next, in each group, the researcher has tested for the increase in BGC using a two-sample Student's t-test for unpaired observations.  
3. A researcher studies the effect of artificial sweeteners on the blood glucose level (BGC) in mice. The researcher has partitioned $N=100$ mice into ten equally-sized groups: nine experimental and one control. Mice in the control group were given a normal diet. In the experimental groups, the mice were given food enriched in glucose; sucralose; aspartam; erythritol; stevia; xylitol; saccharine; acesulfam K; manninol. After four days, the researcher has measured the BGC of all mice. Next, the researcher has used the two-sample Student's t test for unpaired observations to compare the average BGC between the ten groups. To correct for multiple tests, the researcher has used the Benjamini-Hochberg procedure at FDR level 20%.      
4. A researcher studies the effect of salt consumption on the abdominal circumference (AC) in mice. The researcher has partitioned $N=20$ mice into equally-sized experimental and control groups. The AC of all mice was measured at the beginning of the experiment. Mice from the experimental group were given a diet enriched in salt. After a week, the AC was measured again, and the researcher used a two-sample Student's t test for paired observations in each group. The results of the test in each group were insigificant, and the researcher concluded that salt consumption does not increase the abdominal circumference in the span of one week.     

*Acknowledgement.*  
Exercise 3 was partly inspired by the following resource: https://stats.libretexts.org/Bookshelves/Applied_Statistics/Biological_Statistics_(McDonald)/06%3A_Multiple_Tests/6.01%3A_Multiple_Comparisons

<center><img src='https://drive.google.com/uc?export=view&id=12CrUdXDAiltLBT26sG7HZ_HciIhvGyT8'></center>|